In [105]:
from copy import copy
from random import uniform
from math import exp

In [107]:
def generate_table(func, rows, columns):
    """ (Int * Int -> Real) * Int * Int -> [[Real]]
    """
    return [[func(i,j) for j in range(columns)] for i in range(rows)]
def random_weight(i, j):
    """ Int * Int -> Real
    """
    return uniform(-0.05, 0.05)

In [108]:
w = generate_table(random_weight, 2, 3)
w

[[-0.04645844858422835, -0.02036451811546619, -0.027940171639990564],
 [0.032172992822303195, -0.02638316225825742, -0.03323892759135694]]

In [109]:
scale = [3, 2, 2]

In [110]:
weights1 = generate_table(random_weight, scale[1], scale[0] + 1)
weights2 = generate_table(random_weight, scale[2], scale[1] + 1)
weights = [weights1, weights2]
print(weights1, weights2)

[[-0.0008035476506728784, -0.03927806101713141, 0.02075181009818952, -0.022447755306467143], [0.006232984074222181, -0.01562698308087506, -0.012234018534533174, 0.03408603808816228]] [[-0.016468492291871327, 0.027769765568872354, 0.0203272159521755], [-0.015621658989372587, 0.03784564657308806, 0.04108523586756917]]


In [111]:
def initialize_ann(scale):
    ann = {}
    ann['scale'] = scale
    ann['weights'] = [generate_table(random_weight, scale[layer], scale[layer - 1] + 1) for layer in range(1, len(scale))]
    return ann

In [140]:
ann = initialize_ann(scale)
ann['weights']

[[[-0.009684742701485671,
   -0.014139625840989323,
   0.01078854751913004,
   -0.008746372961945158],
  [0.03914139679056321,
   -0.010877377028860734,
   -0.03571026319587,
   0.04635122197039006]],
 [[0.04997997381457851, 0.04419658987675354, 0.042031157886952594],
  [0.016035985852120224, 0.041015730553760885, 0.020442912434241137]]]

In [114]:
inputs = [1, 2, 3]

In [115]:
def output_from_unit(inputs, weights_to_unit):
    """ [Real] * [[Real]] * (Real -> Real) -> Real
    
    >>> len(inputs) + 1 == len(weights_on_unit)
    """
    w = copy(weights_to_unit)
    x = copy(inputs)
    x.insert(0, 1)
    net = sum([x[i] * w[i] for i in range(len(x))])
    return unit_function(net)

In [116]:
output_from_unit(inputs, weights1[0])

0.48352565189701063

In [117]:
def propagate(inputs, propagate_function):
    """ [Real] * ([Real] -> [Real]) -> [Real]
    """
    return propagate_function(inputs)

In [138]:
def forward_propagate(ann):
    """ Ann -> ([Real] -> [[Real]])
    
    output all outputs on all layers.
    """
    def outputs_on_layers(ann_inputs):
        inputs_from_upstream = copy(ann_inputs)
        result = []
        for i in range(1, len(scale)):
            propagate_function = lambda x: [output_from_unit(x, ann['weights'][i - 1][unit]) for unit in range(scale[i])]
            outputs_on_layer = propagate(inputs_from_upstream, propagate_function)
            result += [outputs_on_layer]
            inputs_from_upstream = outputs_on_layer
        return result
    return outputs_on_layers

In [141]:
o = forward_propagate(ann)(inputs)
o

[[0.49287888344637987, 0.5239559338735332],
 [0.5234293327413644, 0.5117385822583407]]

In [133]:
errors = [uniform(-1, 1) for i in range(scale[-1])]
errors_from_upstream = copy(errors)
errors_from_upstream

[0.7444144559846977, 0.9033938852576191]

In [142]:
outputs_on_layer = o[-1]
a = lambda o, k, delta: o[k] * (1 - o[k]) * delta[k]
errors_on_output_layer = [a(outputs_on_layer, unit, errors) for unit in range(scale[-1])]
errors_on_output_layer

[0.18569497986461223, 0.2257239887742236]

In [155]:
outputs_on_layer = o[-2]
b = lambda o, h, delta, w: o[h] * (1 - o[h]) * sum([w[k][h] * delta[k] for k in range(len(delta))])
errors_on_hidden_layer = [b(outputs_on_layer, h, errors_on_output_layer, ann['weights'][-1]) for h in range(scale[-2])]
errors_on_hidden_layer

[0.003224530030147867, 0.0043563066770165556]

In [168]:
def backward_propagate(ann, outputs):
    """ Ann -> ([Real] -> [Real])
    """
    def errors_on_layers(errors):
        errors_from_upstream = copy(errors)
        result = []
        # for output layer:
        f = lambda o, k, delta: o[k] * (1 - o[k]) * delta[k] # eq.(4.14)
        propagate_function = lambda x: [f(outputs[-1], unit, x) for unit in range(scale[-1])]
        errors_on_output_layer = propagate_function(errors)
        result += [errors_on_output_layer]
        # for hidden layers:
        if len(scale) - 2 != 0: # exists hidden layer(s)
            errors_from_upstream = errors_on_output_layer
            for hidden_layer in range(len(scale) - 2):
                f = lambda o, h, delta, w: o[h] * (1 - o[h]) * sum([w[k][h] * delta[k] for k in range(len(delta))]) # eq. (4.15)
                propagate_function = lambda x: [f(outputs[-2 - hidden_layer], unit, x, ann['weights'][-1 - hidden_layer]) for unit in range(scale[-2 - hidden_layer])]
                errors_on_hidden_layer = propagate_function(errors_from_upstream)
                result.insert(0, errors_on_hidden_layer)
                errors_from_upstream = errors_on_hidden_layer
        return result
    return errors_on_layers

In [169]:
outputs = forward_propagate(ann)(inputs)
backward_propagate(ann, outputs)(errors)

[[0.003224530030147867, 0.0043563066770165556],
 [0.18569497986461223, 0.2257239887742236]]

In [181]:
def copy_ann(ann):
    """ Ann -> Ann
    """
    result = {}
    result['scale'] = ann['scale']
    result['weights'] = copy(ann['weights'])
    return result

In [188]:
def update_weights(ann, inputs, targets, learning_speed):
    """ Ann * [Real] * [Real] -> Ann
    """
    def generate_x(inputs, outputs):
        # layer = 0
        x = [[inputs[f] for f in range(scale[0])] for t in range(scale[1])]
        # 
    result = copy_ann(ann)
    outputs = forward_propagate(ann)(inputs)
    errors = [targets[i] - outputs[-1][i] for i in range(len(targets))]
    delta = backward_propagate(ann, outputs)(errors)
    x = generate_x(inputs, outputs)
    for l in range(len(scale) - 1):
        for t in range(scale[l + 1]):
            for f in range(scale[l]):
                delta_w = learning_speed * delta[l] * x[l][t][f]
                result['weights'][l][t][f] = ann['weights'][l][t][f] + delta_w  
    return result

In [191]:
o = forward_propagate(ann)(inputs)
errors = [targets[i] - outputs[-1][i] for i in range(len(targets))]
delta = backward_propagate(ann, o)(errors)

In [193]:
x= [inputs]
for l in range(len(scale) - 1)
x

[[1, 2, 3], [0.49287888344637987, 0.5239559338735332]]

In [189]:
targets = errors
learning_speed = 0.01
update_weights(ann, inputs, targets, learning_speed)

{'scale': [3, 2, 2],
 'weights': [[[-0.009684742701485671,
    -0.014139625840989323,
    0.01078854751913004,
    -0.008746372961945158],
   [0.03914139679056321,
    -0.010877377028860734,
    -0.03571026319587,
    0.04635122197039006]],
  [[0.04997997381457851, 0.04419658987675354, 0.042031157886952594],
   [0.016035985852120224, 0.041015730553760885, 0.020442912434241137]]]}